# Module import

In [1]:
import sys
sys.path.append(r'../Script/')
from ETGEMs_function import *

# Data initialization

In [2]:
import pandas as pd
import cobra

reaction_g0_file='./Basic Data/reaction_g0.txt'
metabolites_lnC_file = './Basic Data/metabolites_lnC.txt'
reaction_kcat_MW_file='./Basic Data/ID_kcat_MW_file.csv'

model=cobra.io.read_sbml_model('./Basic Data/iML1515.xml')
cobra.manipulation.modify.convert_to_irreversible(model)

model.reactions.get_by_id('ATPM').bounds = (0.0, 1000)

demand = model.add_boundary(model.metabolites.ser__L_c, type="demand") 

Concretemodel_Need_Data=Get_Concretemodel_Need_Data(reaction_g0_file,metabolites_lnC_file,model,reaction_kcat_MW_file)

# 1.MDF calculation
Preset a lower growth rate, for example, when set "product_value = 0.1", and then get an MDF value (9.864).

In [3]:
product_id = 'DM_ser__L_c'
#product_id='DM_4hpro_LT_c'
#product_id='DM_ser__L_c'
# biomass_id='BIOMASS_Ec_iML1515_core_75p37M'
E_total = 0.13 #eaual to e_pool （0.228）* saturation（0.5）
substrate_name = 'EX_glc__D_e_reverse'
substrate_value = 10
product_value = 0.1
K_value=1249

B_value=MDF_Calculation(Concretemodel_Need_Data,product_value,product_id,substrate_name,substrate_value,K_value,E_total,'gurobi')
print("B value : " +str(B_value))

B value : 9.864359490782546


# 2.Maximum growth rate calculation
By taking the MDF above obtained as the lower bound of thermodynamic constraints.

In [4]:
obj_name='DM_ser__L_c'
obj_target='maximize'
E_total=0.13
substrate_name='EX_glc__D_e_reverse'
substrate_value=10
K_value=1249

max_product_under_mdf=Max_Growth_Rate_Calculation(Concretemodel_Need_Data,obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,B_value,'gurobi')
print("Max product value : " +str(max_product_under_mdf))

Max product value : 5.681431109644585


# 3.Minimum enzyme cost calculation
By fixing the MDF and maximum growth rate above obtained.

In [5]:
product_id='DM_ser__L_c'
E_total=0.13 #eaual to e_pool （0.228）* saturation（0.5）
substrate_name='EX_glc__D_e_reverse'
substrate_value=10
product_value=max_product_under_mdf
K_value=1249
B_value=B_value
#B_value=0

min_E=Min_Enzyme_Cost_Calculation(Concretemodel_Need_Data,product_value,product_id,substrate_name,substrate_value,K_value,E_total,B_value,'gurobi')
print("Min enzyme amount : " +str(min_E))

Min enzyme amount : 0.1299999999999942


# 4.Minimum flux sum calculation（pFBA）
Used to simplify the output file below

In [6]:
product_id='DM_ser__L_c'
E_total=0.13 #eaual to e_pool （0.228）* saturation（0.5）
substrate_name='EX_glc__D_e_reverse'
substrate_value=10
product_value=max_product_under_mdf
K_value=1249
B_value=B_value
#B_value=B_0

[min_V,Concretemodel]=Min_Flux_Sum_Calculation(Concretemodel_Need_Data,product_value,product_id,substrate_name,substrate_value,K_value,E_total,B_value,'gurobi')
print("Min flux amount : " +str(min_V))

Min flux amount : 465.3488457712224


# 5.Pathway information output
It is used to extract the following various lists.

In [7]:
model=Concretemodel_Need_Data['model']
reaction_kcat_MW=Concretemodel_Need_Data['reaction_kcat_MW']
reaction_g0=Concretemodel_Need_Data['reaction_g0']
coef_matrix=Concretemodel_Need_Data['coef_matrix']
metabolite_list=Concretemodel_Need_Data['metabolite_list']
use_result = Get_Results_Thermodynamics(model,Concretemodel,reaction_kcat_MW,reaction_g0,coef_matrix,metabolite_list)
use_result = use_result[use_result['flux'] > 1e-10] 
use_result = use_result.sort_values(by = 'flux',axis = 0,ascending = False)
use_result["reaction"] = use_result.apply(lambda row: model.reactions.get_by_id(row.name).reaction, axis = 1)
use_result["gpr"] = use_result.apply(lambda row: model.reactions.get_by_id(row.name).gene_reaction_rule, axis = 1)
use_result.to_csv('./Analysis Result/ser_pathway/' + 'ser__L_c' + str(round(max_product_under_mdf,3)) + '_' + str(round(B_value,3)) + '_' + str(round(min_E,3)) + 'ser_EcoETM-pathway-glc10-1.csv', sep=',', header=True, index=True,mode='w')

# 6.List extraction of candidate bottleneck reactions
Standard: the thermodynamic driving force (f) is equal to MDF value (B) above mentiond

In [8]:
#若想输出所有反应的mdf范围，隔一个运行一个
use_result_tmp=use_result[use_result['f']>-1249]
use_result_select=use_result_tmp[abs(use_result_tmp['f']-B_value)<= 1249]
use_result_select.head(150)

,flux,z,f,enz,met_concentration,reaction,gpr
G6PDH2r,14.203578,1.0,9.864359,1.417924e-03,;h_c : 1.0;g6p_c : 3.050925838678005e-05;6pgl_...,g6p_c + nadp_c --> 6pgl_c + h_c + nadph_c,b1852
PGL,14.203578,1.0,9.864359,7.785293e-04,;h2o_c : 1.0;h_c : 1.0;6pgc_c : 0.000111091951...,6pgl_c + h2o_c --> 6pgc_c + h_c,b0767
MDH2,11.362862,1.0,57.684900,1.604093e-02,;q8h2_c : 4.999999992621096e-07;mal__L_c : 4.9...,mal__L_c + q8_c --> oaa_c + q8h2_c,b2210
GND,9.116715,1.0,9.864359,5.926014e-03,;nadph_c : 0.0029023904047720188;ru5p__D_c : 0...,6pgc_c + nadp_c --> co2_c + nadph_c + ru5p__D_c,b2029
HEX1,8.125768,1.0,15.135641,5.932189e-03,;adp_c : 4.999999992621078e-07;g6p_c : 3.05092...,atp_c + glc__D_c --> adp_c + g6p_c + h_c,b2388
GAPP,7.531199,1.0,26.854350,2.565879e-03,;h2o_c : 1.0;glyald_c : 1.6155739644947679e-06...,g3p_c + h2o_c --> glyald_c + pi_c,b1727
PGI_reverse,6.077810,1.0,9.864359,4.946646e-04,;f6p_c : 0.0005242179147187425;g6p_c : 3.05092...,f6p_c --> g6p_c,b4025
RPE,6.077810,1.0,9.864359,6.377597e-05,;xu5p__D_c : 0.0016310028219376832;ru5p__D_c :...,ru5p__D_c --> xu5p__D_c,b3386
ACONTb,5.681431,1.0,9.864359,1.604924e-03,;h2o_c : 1.0;acon_C_c : 1.7466896272928242e-05...,acon_C_c + h2o_c --> icit_c,b0118 or b1276
CS,5.681431,1.0,12.668555,2.612969e-03,;h_c : 1.0;oaa_c : 5.910497757229258e-05;h2o_c...,accoa_c + h2o_c + oaa_c --> cit_c + coa_c + h_c,b0720


In [9]:
path_reac_list=list(use_result_select.index)

# 7.Determination of bottleneck reaction
Calculate the maximum thermodynamic driving force for reactions in above list, if the value is still equal to B, it is the bottleneck reaction.

In [10]:
import pandas as pd
import numpy as np
import datetime
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo.environ as pyo
from concurrent.futures import ProcessPoolExecutor, as_completed
starttime = datetime.datetime.now()
max_min_Df_list_fixed=pd.DataFrame()

with ProcessPoolExecutor() as executor:
    futures = {executor.submit(Get_Max_Min_Df_Complete,Concretemodel_Need_Data,eachreaction,'maximize',K_value,B_value,max_product_under_mdf,\
                               product_id,E_total,substrate_name,substrate_value,'gurobi'): eachreaction for eachreaction in path_reac_list}
    for future in as_completed(futures):
        tmp = future.result()
        for eachindex in tmp.index:
            print(eachindex,tmp.loc[eachindex,'max_value'])
            max_min_Df_list_fixed.loc[eachindex,'max_Df_complete']=tmp.loc[eachindex,'max_value']

endtime = datetime.datetime.now()
print (endtime - starttime)

max_min_Df_list_fixed.to_csv('./Analysis Result/ser_pathway/max_min_Df_complete_for_all_reaction_ser_glc10_1.csv', sep=',', header=True, index=True,mode='w')

MDH2 95.28763057976799
HEX1 42.46436149078254
G6PDH2r 11.55945530103298
PGL 13.357537817774034
GND 13.357537817774094
CS 43.28490213415385
MALS 60.238370954831296
ACONTb 9.864361490782443
GAPP 50.473662438666345
PFL 75.30545889122811
PGI_reverse 11.559455299777042
ICL 28.717830311459917
FUM 30.728720981565004
RPE 12.810096808994496
ACONTa 9.864361490782432
SUCDi 61.3178323114597
HSDy_reverse 47.532931175954744
ACALD 42.97798060650166
GHMT2r_reverse 36.46708993639643
ASPK 14.867087936396654
THRA 54.120558757073916
HSK 66.56708793639666
THRS 92.94654929302507
ASPTA_reverse 29.887628579768226
MTHFD_reverse 25.60272844561409
FTHFLi 36.64109740044561
MTHFC_reverse 21.528720981565005
GLUDy_reverse 29.88762857976809
ASAD_reverse 30.458923711905793
EDD 58.730736318212074
EDA 44.88218980224249
TALA 13.254551108771611
TKT2 13.254551108771677
LALDO3 42.06708793639665
LCADi 84.76708793639665
RPI_reverse 15.755834127206366
TKT1 15.755834129063471
TPI_reverse 15.755834127206306
L_LACD2 111.857441963

In [11]:
max_min_Df_list_fixed=max_min_Df_list_fixed.sort_values(by='max_Df_complete',ascending = True)
max_min_Df_list_fixed.head(150)

,max_Df_complete
ACONTa,9.864361
ACONTb,9.864361
PGI_reverse,11.559455
G6PDH2r,11.559455
RPE,12.810097
TALA,13.254551
TKT2,13.254551
PGL,13.357538
GND,13.357538
ASPK,14.867088


In [12]:
Bottleneck_reaction=max_min_Df_list_fixed[(max_min_Df_list_fixed['max_Df_complete']-B_value)<=0.001]
Bottleneck_reaction

,max_Df_complete
ACONTa,9.864361
ACONTb,9.864361


In [13]:
use_result_select.loc[Bottleneck_reaction.index[0],'met_concentration']

';h2o_c : 1.0;cit_c : 0.019999984498682274;acon_C_c : 1.7466896272928242e-05'

# 8.List extraction of candidate limiting metabolites
Standard: involved in bottleneck reactions, except fo water (h2o_c) and protons (h_c).

In [14]:
Bottleneck_reaction_lsit=list(Bottleneck_reaction.index)
Bottleneck_reaction_met=[]
for rea in model.reactions:
    if rea.id in Bottleneck_reaction_lsit:
        #print(rea)
        for met in model.metabolites:
            try:
                rea.get_coefficient(met.id)  
            except:
                pass
            else:
                if met.id !='h_c' and met.id !='h2o_c':
                    Bottleneck_reaction_met.append(met.id)
                

Bottleneck_reaction_met=list(set(Bottleneck_reaction_met))
Bottleneck_reaction_met

['icit_c', 'cit_c', 'acon_C_c']

# 9.Determination of limitting metabolites
Calculate the maximum and minimum concentrations for metabolites in above list, if the two values are equal, it is the limiting metabolite.

In [15]:
import pandas as pd
import numpy as np
import datetime
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo.environ as pyo
from concurrent.futures import ProcessPoolExecutor, as_completed
starttime = datetime.datetime.now()
max_min_concentration_list_fixed = pd.DataFrame()

with ProcessPoolExecutor() as executor:
    futures = {executor.submit(Get_Max_Min_Met_Concentration,Concretemodel_Need_Data,eachmet,'maximize',K_value,B_value,\
        max_product_under_mdf,product_id,E_total,substrate_name,substrate_value,list(Bottleneck_reaction.index),'gurobi'): eachmet for eachmet in Bottleneck_reaction_met}
    for future in as_completed(futures):
        tmp = future.result()
        for eachindex in tmp.index:
            #print(eachindex,tmp.loc[eachindex,'max_value'])
            max_min_concentration_list_fixed.loc[eachindex,'max_concentration'] = tmp.loc[eachindex,'max_value']

with ProcessPoolExecutor() as executor:
    futures = {executor.submit(Get_Max_Min_Met_Concentration,Concretemodel_Need_Data,eachmet,'minimize',K_value,B_value,\
        max_product_under_mdf,product_id,E_total,substrate_name,substrate_value,list(Bottleneck_reaction.index),'gurobi'): eachmet for eachmet in Bottleneck_reaction_met}
    for future in as_completed(futures):
        tmp = future.result()
        for eachindex in tmp.index:
            #print(eachindex,tmp.loc[eachindex,'max_value'])
            max_min_concentration_list_fixed.loc[eachindex,'min_concentration'] = tmp.loc[eachindex,'min_value']
            
endtime = datetime.datetime.now()
print (endtime - starttime)
max_min_concentration_list_fixed.to_csv('./Analysis Result/ser_pathway/max_min_concentration_for_specific_metabolite_ser_glc10-1.csv', sep=',', header=True, index=True,mode='w')


0:00:10.745123


In [16]:
max_min_concentration_list_fixed

,max_concentration,min_concentration
cit_c,-3.912023,-3.912024
icit_c,-14.508657,-14.508658
acon_C_c,-10.955202,-10.955203


In [17]:
Limiting_metabolite = max_min_concentration_list_fixed[(max_min_concentration_list_fixed['max_concentration'] - max_min_concentration_list_fixed['min_concentration']) <= 0.001]
Limiting_metabolite

,max_concentration,min_concentration
cit_c,-3.912023,-3.912024
icit_c,-14.508657,-14.508658
acon_C_c,-10.955202,-10.955203


# 10.List extraction of candidate key enzymes
Standard: The amount of enzyme usage was more than 0.0015 mg/gDW （above 1% of e_pool）

In [18]:
step5_file = pd.read_csv('./Analysis Result/ser_pathway/' + 'ser__L_c' + str(round(max_product_under_mdf,3)) + '_' + str(round(B_value,3)) + '_' + str(round(min_E,3)) + 'ser_EcoETM-pathway-glc10-1.csv',index_col=0)
step5_file_sort = step5_file.sort_values(by='enz',ascending = False)
step5_file_sort.head(10)

,flux,z,f,enz,met_concentration,reaction,gpr
THRS,5.681431,1.0,17.694543,0.018588,;h2o_c : 1.0;thr__L_c : 0.02000000000856292;pi...,h2o_c + phom_c --> pi_c + thr__L_c,b0004
MDH2,11.362862,1.0,57.684900,0.016041,;q8h2_c : 4.999999992621096e-07;mal__L_c : 4.9...,mal__L_c + q8_c --> oaa_c + q8h2_c,b2210
ASAD_reverse,5.681431,1.0,9.864359,0.010025,;h_c : 1.0;nadph_c : 0.0029023904047720188;nad...,4pasp_c + h_c + nadph_c --> aspsa_c + nadp_c +...,b3433
GHMT2r_reverse,5.681431,1.0,9.864359,0.007027,;h2o_c : 1.0;thf_c : 0.015093006405025707;gly_...,gly_c + h2o_c + mlthf_c --> ser__L_c + thf_c,b2551
EDD,5.086863,1.0,29.706920,0.006804,;h2o_c : 1.0;2ddg6p_c : 0.02000000000856292;6p...,6pgc_c --> 2ddg6p_c + h2o_c,b1851
MTHFD_reverse,5.681431,1.0,9.864359,0.006125,;nadp_c : 0.00029023904047720186;nadph_c : 0.0...,methf_c + nadph_c --> mlthf_c + nadp_c,b0529
HEX1,8.125768,1.0,15.135641,0.005932,;adp_c : 4.999999992621078e-07;g6p_c : 3.05092...,atp_c + glc__D_c --> adp_c + g6p_c + h_c,b2388
GND,9.116715,1.0,9.864359,0.005926,;nadph_c : 0.0029023904047720188;ru5p__D_c : 0...,6pgc_c + nadp_c --> co2_c + nadph_c + ru5p__D_c,b2029
NADH16pp,6.275999,1.0,-9999.000000,0.004844,;h_c : 1.0;h_p : 1.0;nadh_c : 4.99999999262107...,4.0 h_c + nadh_c + q8_c --> 3.0 h_p + nad_c + ...,b2280 and b2281 and b2287 and b2288 and b2282 ...
MALS,5.681431,1.0,25.789093,0.003955,;h_c : 1.0;h2o_c : 1.0;mal__L_c : 4.9999999926...,accoa_c + glx_c + h2o_c --> coa_c + h_c + mal_...,b4014 or b2976


In [19]:
enz_use_reaction_list = list(step5_file_sort[step5_file_sort['enz'] > 0.0000013].index)
#enz_use_reaction_list

# 11.Determination of key enzymes.
Calculate the minimum enzyme cost for reactions in above list, and the higher the value, the more critical (key) the enzyme.

In [20]:
import pandas as pd
import numpy as np
import datetime
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo.environ as pyo
from concurrent.futures import ProcessPoolExecutor, as_completed
starttime = datetime.datetime.now()
max_min_E_list_fixed = pd.DataFrame()

with ProcessPoolExecutor() as executor:
    futures = {executor.submit(Get_Max_Min_E,Concretemodel_Need_Data,eachreaction,'maximize',K_value,B_value,\
                    max_product_under_mdf,product_id,E_total,substrate_name,substrate_value,'gurobi'): eachreaction for eachreaction in enz_use_reaction_list}
    for future in as_completed(futures):
        tmp = future.result()
        for eachindex in tmp.index:
            #print(eachindex,tmp.loc[eachindex,'max_value'])
            max_min_E_list_fixed.loc[eachindex,'max_E']=tmp.loc[eachindex,'max_value']

with ProcessPoolExecutor() as executor:
    futures = {executor.submit(Get_Max_Min_E,Concretemodel_Need_Data,eachreaction,'minimize',K_value,B_value,\
                    max_product_under_mdf,product_id,E_total,substrate_name,substrate_value,'gurobi'): eachreaction for eachreaction in enz_use_reaction_list}
    for future in as_completed(futures):
        tmp = future.result()
        for eachindex in tmp.index:
            #print(eachindex,tmp.loc[eachindex,'max_value'])
            max_min_E_list_fixed.loc[eachindex,'min_E']=tmp.loc[eachindex,'min_value']
endtime = datetime.datetime.now()
print (endtime - starttime)
max_min_E_list_fixed.to_csv('./Analysis Result/ser_pathway/max_min_E_ser_glc10-1.csv', sep=',', header=True, index=True,mode='w')


0:01:15.155416


In [21]:
max_min_E_list_fixed.sort_values(by='max_E',ascending = False)

,max_E,min_E
THRS,0.018588,0.018588
MDH2,0.016041,0.016041
ASAD_reverse,0.010025,0.010025
GHMT2r_reverse,0.007027,0.007027
EDD,0.006804,0.006804
MTHFD_reverse,0.006125,0.006125
HEX1,0.005932,0.005932
GND,0.005926,0.005926
NADH16pp,0.004844,0.004844
MALS,0.003955,0.003955


# Congratulations！

Now you have completed all the calculations at the first turning point. 
To obtain all the turning point data, 
you need to go back to "1. MDF calculation" 
and change the "product_value=0.1" to "product_ Value=5.7" (a value slightly larger than the product_value you just calculated), 
you will get the second turning point data. 

Note：
(1) If the new "product_value" is set too high, you may miss some turning point date, but don't worry, the solver gurobi has good differentiation, and a slight increase is the best choice (only add 1‰ or 1% than "product_value" of last round often enough).
(2) Don't forget to change the suffix of each output file name, such as -1, -2, -3, etc.

In the manuscript "Improving pathway prediction accuracy of constraints-based metabolic network models by treating enzymes as microcompartments", 
all the 13 "product_value"s used by the authors are: 0.1, 5.7, 5.92, 5.95, 8.31, 8.4, 8.92, 9, 17.2, 20.1, 20.13, 20.67 and 20.84, respectively, 
for reference.